In [ ]:
def all_in_one_N(your_json):
    import json
    import datetime
    import seaborn
    import numpy
    import math
    import pandas as pd
    from matplotlib import pyplot
    import matplotlib
    matplotlib.rcParams.update({'font.size': 48})

    stations = {}
    for n_lines, line in enumerate(open(your_json)):
        for vehicle in json.loads(line.strip()):
            if vehicle.get('current_status') != 1: # STOPPED_AT
                continue
            try:
                line = vehicle['trip']['route_id'] #.rstrip('X') # fold express into normal
                if line not in ['G', '1', '2', '3', '4', '5', '5X', '6', '6X', '7', '7X', 'N', 'Q', 'R', 'W', 'A', 'C', 'E', 'B', 'D', 'F', 'M', 'L', 'J', 'H']:
                    print 'weird line', line
                    continue
                if 'stop_id' in vehicle:
                    stop = vehicle['stop_id']
                else:
                    # L and SI stop at every station, need to use 
                    stop = '%d%s' % (vehicle['current_stop_sequence'], vehicle['trip']['trip_id'][-1])
                key = (line, stop)
                timestamp = vehicle['timestamp'] # datetime.datetime.utcfromtimestamp(vehicle['timestamp'])
                stations.setdefault(key, set()).add(timestamp)
            except:
                print 'weird vehicle', vehicle
                continue

        #if n_lines >= 10000:
        #    break
    
    name = your_json #.astype(str)
    name = name[:-5]
    name = name[4:]
    pyplot.figure(figsize=(10, 10))

    # Look at all intervals between subway arrivals
    def next_whole_minute(t):
        return t+59 - (t+59)%60

    deltas = []
    next_subway = []
    next_subway_by_time_of_day = [[] for x in xrange(24 * 60)]
    next_subway_by_line_ts = []
    next_subway_by_line_ls = []
    next_subway_rush_hour = []
    time_ = []
    max_limit = 4 * 3600 # cap max value so that Seaborn's KDE works better
    for key, values in stations.iteritems():
        line, stop = key
        values = sorted(values)
        print key, len(values)
        last_value = None
        for i in xrange(1, len(values)):
            last_value, value = values[i-1], values[i]
            if value - last_value >= max_limit:
                continue
            deltas.append(1. / 60 * (value - last_value))
            for t in xrange(next_whole_minute(last_value), value, 60):
                x = (t // 60 + 19 * 60) % (24 * 60) # 19 from UTC offset
                waiting_time = 1. / 60 * (value - t)
                time_.append(t)
                next_subway_by_time_of_day[x].append(waiting_time)
                next_subway.append(waiting_time)
                next_subway_by_line_ts.append(waiting_time)
                next_subway_by_line_ls.append(line)
                if x >= 7 * 60 and x < 19 * 60:
                    next_subway_rush_hour.append(waiting_time)
    for data, fn, title, color in [(deltas, 'time_between_arrivals_'+ name, 'Distribution of delays between subway arrivals_' +name, 'blue'),
                                       (next_subway, 'time_to_next_arrival_' + name, 'Distribution of time until the next subway arrival_' + name, 'red')]:
        print 'got', len(data), 'points'
        pyplot.clf()
        lm = seaborn.distplot(data, bins=numpy.linspace(0, 60, num=61), color=color, kde_kws={'gridsize': 2000})
        pyplot.xlim([-1, 40])
        pyplot.title(title , fontsize = 10)
        pyplot.xticks(fontsize = 10)
        pyplot.yticks(fontsize = 10)
        pyplot.xlabel('Time (min)', fontsize = 10)
        pyplot.ylabel('Probability distribution', fontsize = 10)
        pyplot.savefig(fn)
        print 'mean', 60*numpy.mean(data), 'median', 60*numpy.median(data)
        
        
    # Plot deltas by line
    pyplot.clf()
    seaborn.violinplot(orient='h',
                       x=next_subway_by_line_ts,
                       y=next_subway_by_line_ls,
                       order=['5', '5X', '7', '7X', 'N', 'W', 'F',  'G'],
                       scale='width',
                       palette=['#EE352E']*3 + ['#00933C']*3 + ['#808183', '#A7A9AC', '#555555'],
                       bw=0.03, cut=0, gridsize=2000)
    pyplot.xlim([-1, 40])
    pyplot.title('Time until the next subway ' +name , fontsize = 10 )
    pyplot.title('Time until the next subway ' +name , fontsize = 10 )
    pyplot.xlabel('Time (min)' , fontsize = 10 )
    pyplot.ylabel('Line' , fontsize = 10 )
    pyplot.xticks(fontsize = 10)
    pyplot.yticks(fontsize = 10)
    fq = 'time_to_arrival_by_line_' + name 
    pyplot.savefig(fq)
    
    #make dataframe
    x = next_subway_by_line_ts
    y = next_subway_by_line_ls
    z = time_


    df = pd.DataFrame({'time_to_next_subway':x, 'line':y, 'time':z})
    csvname = 'subway_gaps_' + name
    df.to_csv(csvname, index=False)

    

In [ ]:
#insert your json
all_in_one_N('YOUR_JSON')